In [93]:
!pip install pillow
!pip install sem-meta

In [135]:
from PIL import Image
from sem_meta import SEMMeta # this library seems still under development

In [151]:
import os
print(os.getcwd())


/Users/vyshnavikota


In [177]:
with Image.open("Atest/Acquire BF.tif") as im:
  meta,tags=SEMMeta.ImageMetadata(im)

In [179]:
#where is the metadata
dict(meta)

{256: (2048,),
 257: (2115,),
 258: (8,),
 259: (1,),
 262: (1,),
 273: (174,),
 278: (2115,),
 279: (4331520,),
 282: ((10, 1),),
 283: ((10, 1),),
 296: (3,),
 65450: ((1684684800, 1000),),
 65451: ((1684684800, 1000),),
 65452: (3,),
 34682: ('<Root><Data><Label>Microscope</Label><Value>Microscope Titan200 200 kV D3362 AnalyticalSuperTwin</Value><Unit></Unit></Data><Data><Label>User</Label><Value>SUPERVISOR</Value><Unit></Unit></Data><Data><Label>Gun type</Label><Value>FEG</Value><Unit></Unit></Data><Data><Label>High tension</Label><Value>200</Value><Unit>kV</Unit></Data><Data><Label>Extraction voltage</Label><Value>4000</Value><Unit>V</Unit></Data><Data><Label>Gun lens</Label><Value>1</Value><Unit></Unit></Data><Data><Label>Emission</Label><Value>402.0</Value><Unit>uA</Unit></Data><Data><Label>Mode</Label><Value> STEM nP SA Zoom Diffraction</Value><Unit></Unit></Data><Data><Label>Defocus</Label><Value>-0.102</Value><Unit>um</Unit></Data><Data><Label>Magnification</Label><Value>5000

In [115]:
#indexing key '34682'
dict(meta)[34682][0]#unfortunately, the output is in .xml format, you need to convert it

'[User]\r\nDate=07/26/2017\r\nTime=06:03:01 PM\r\nUser=Supervisor\r\nUserText=Helios\r\nUserTextUnicode=480065006C0069006F007300\r\n\r\n[System]\r\nType=DualBeam\r\nDnumber=9924093\r\nSoftware=10.1.0.5054\r\nBuildNr=5054\r\nSource=FEG\r\nColumn=Elstar\r\nFinalLens=Elstar\r\nChamber=xT-SDB\r\nStage=110 x 110\r\nPump=TMP\r\nESEM=no\r\nAperture=AVA\r\nScan=PIA 3.0\r\nAcq=PIA 3.0\r\nEucWD=0.00402742\r\nSystemType=Helios NanoLab G3 CX\r\nDisplayWidth=0.518\r\nDisplayHeight=0.324\r\n\r\n[Beam]\r\nHV=30000\r\nSpot=1\r\nStigmatorX=-0.126472\r\nStigmatorY=0.0515258\r\nBeamShiftX=-1.07812e-005\r\nBeamShiftY=-1.79167e-005\r\nScanRotation=0\r\nImageMode=\r\nFineStageBias=0\r\nBeam=IBeam\r\nScan=IScan\r\n\r\n[IBeam]\r\nSource=Ga LMIS\r\nColumnType=Tomahawk\r\nFinalLens=Tomahawk\r\nAcq=PIA 3.0\r\nAperture=AVA\r\nApertureDiameter=1.33e-007\r\nHV=30000\r\nHFW=8.28867e-005\r\nVFW=5.52578e-005\r\nWD=0.0129585\r\nBeamCurrent=2.49492e-009\r\nTiltCorrectionIsOn=\r\nDynamicFocusIsOn=\r\nDynamicWDIsOn=\r\nSc

In [109]:
import xml.etree.ElementTree as ET
import json

# Assuming meta is already defined in your environment
xml_string = dict(meta)[34682][0]

# Parse XML
root = ET.fromstring(xml_string)

data_dict = {}
stage_dict = {}
filter_dict = {}

for data in root.findall("Data"):
    label = (data.find("Label").text or "").strip()
    value = (data.find("Value").text or "").strip()
    unit = (data.find("Unit").text or "") if data.find("Unit") is not None else ""
    unit = unit.strip()

    # Try converting value to number
    try:
        if "." in value or "e" in value.lower():
            value = float(value)
        else:
            value = int(value)
    except ValueError:
        pass

    # --- Grouping rules ---
    if label.startswith("Stage "):
        axis = label.split("Stage ")[1]
        stage_dict[axis] = {"Value": value, "Unit": unit} if unit else value

    elif label.startswith("Filter "):
        subkey = label.replace("Filter ", "").lower()
        # normalize unit-containing values like "2.5 mm"
        if " " in str(value) and not unit:
            try:
                num, u = value.split()
                value = float(num)
                filter_dict[subkey] = {"Value": value, "Unit": u}
            except Exception:
                filter_dict[subkey] = value
        else:
            filter_dict[subkey] = {"Value": value, "Unit": unit} if unit else value

    elif "detectors" in label.lower():
        data_dict[label] = [int(v.strip()) for v in value.split(";") if v.strip()]

    else:
        # normalize "2.5 mm" style values too
        if " " in str(value) and not unit:
            try:
                num, u = value.split()
                value = float(num)
                data_dict[label] = {"Value": value, "Unit": u}
            except Exception:
                data_dict[label] = value
        else:
            data_dict[label] = {"Value": value, "Unit": unit} if unit else value

# Attach grouped dicts
if stage_dict:
    data_dict["Stage"] = stage_dict
if filter_dict:
    data_dict["Filter"] = filter_dict

# Convert to JSON
json_output = json.dumps(data_dict, indent=2)
print(json_output)


ParseError: syntax error: line 1, column 0 (<string>)

In [59]:
with Image.open("AcquireHAADF.tif") as im:
  meta,tags=SEMMeta.ImageMetadata(im)

In [61]:
dict(meta)

{256: (2048,),
 257: (2115,),
 258: (8,),
 259: (1,),
 262: (1,),
 273: (174,),
 278: (2115,),
 279: (4331520,),
 282: ((10, 1),),
 283: ((10, 1),),
 296: (3,),
 65450: ((1684684800, 1000),),
 65451: ((1684684800, 1000),),
 65452: (3,),
 34682: ('<Root><Data><Label>Microscope</Label><Value>Microscope Titan200 200 kV D3362 AnalyticalSuperTwin</Value><Unit></Unit></Data><Data><Label>User</Label><Value>SUPERVISOR</Value><Unit></Unit></Data><Data><Label>Gun type</Label><Value>FEG</Value><Unit></Unit></Data><Data><Label>High tension</Label><Value>200</Value><Unit>kV</Unit></Data><Data><Label>Extraction voltage</Label><Value>4000</Value><Unit>V</Unit></Data><Data><Label>Gun lens</Label><Value>1</Value><Unit></Unit></Data><Data><Label>Emission</Label><Value>402.0</Value><Unit>uA</Unit></Data><Data><Label>Mode</Label><Value> STEM nP SA Zoom Diffraction</Value><Unit></Unit></Data><Data><Label>Defocus</Label><Value>-0.102</Value><Unit>um</Unit></Data><Data><Label>Magnification</Label><Value>5000

In [57]:
dict(meta)[50431][0]

94

In [37]:
#what the method ImageMetadata actually do
def ImageMetadata(self, img):
    """
    Extracts raw metadata and tag identifiers including 34118
    from a SEM image.

    Parameters:
        img (PIL.Image.Image): The opened SEM image object.

    Returns:
        tuple:
            - image_metadata (dict): Dictionary of available EXIF tags and their values.
            - image_tags (np.ndarray): Array of tag identifiers (keys) present in the image.
    """

    # Retrieve the raw EXIF tag dictionary from the image
    self.image_metadata = img.tag

    # Convert tag keys to a NumPy array for efficient lookup and processing
    self.image_tags = np.array(self.image_metadata)
    return self.image_metadata, self.image_tags

In [65]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from PIL.TiffTags import TAGS

try:
    import SEMMeta  # assuming you have SEMMeta installed
except ImportError:
    SEMMeta = None


def extract_metadata(image_path: str) -> str:
    """
    Extract TIFF + Instrument metadata from a SEM .tif file
    and return as a JSON string with proper naming.
    """
    with Image.open(image_path) as im:
        if SEMMeta is None:
            raise ImportError("SEMMeta module is required for this function.")
        
        meta, tags = SEMMeta.ImageMetadata(im)

        # 1. Map TIFF tag IDs to names
        meta_named = {TAGS.get(k, str(k)): v for k, v in dict(meta).items()}

        # 2. Parse Instrument XML metadata (tag 34682)
        instrument_meta = {}
        xml_data = dict(meta).get(34682)
        if xml_data:
            xml_data = xml_data[0]  # tuple → string
            root = ET.fromstring(xml_data)

            for data in root.findall(".//Data"):
                label = data.find("Label").text.strip()
                value = data.find("Value").text.strip()
                unit_elem = data.find("Unit")
                unit = unit_elem.text.strip() if unit_elem is not None and unit_elem.text else ""

                if unit:
                    instrument_meta[label] = {"value": value, "unit": unit}
                else:
                    instrument_meta[label] = {"value": value}

        # 3. Merge results
        final_meta = {
            "TIFF_Metadata": meta_named,
            "Instrument_Metadata": instrument_meta
        }

        # 4. Convert to JSON string
        return json.dumps(final_meta, indent=2)


In [71]:
json_output = extract_metadata("Acquire BF.tif")
print(json_output)


{
  "TIFF_Metadata": {
    "ImageWidth": [
      2048
    ],
    "ImageLength": [
      2115
    ],
    "BitsPerSample": [
      8
    ],
    "Compression": [
      1
    ],
    "PhotometricInterpretation": [
      1
    ],
    "StripOffsets": [
      174
    ],
    "RowsPerStrip": [
      2115
    ],
    "StripByteCounts": [
      4331520
    ],
    "XResolution": [
      [
        10,
        1
      ]
    ],
    "YResolution": [
      [
        10,
        1
      ]
    ],
    "ResolutionUnit": [
      3
    ],
    "65450": [
      [
        1684684800,
        1000
      ]
    ],
    "65451": [
      [
        1684684800,
        1000
      ]
    ],
    "65452": [
      3
    ],
    "34682": [
      "<Root><Data><Label>Microscope</Label><Value>Microscope Titan200 200 kV D3362 AnalyticalSuperTwin</Value><Unit></Unit></Data><Data><Label>User</Label><Value>SUPERVISOR</Value><Unit></Unit></Data><Data><Label>Gun type</Label><Value>FEG</Value><Unit></Unit></Data><Data><Label>High tension

In [89]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from PIL.TiffTags import TAGS

try:
    import SEMMeta
except ImportError:
    SEMMeta = None


def extract_metadata(image_path: str) -> dict:
    """
    Extract TIFF + Instrument metadata from a SEM .tif file
    and return a clean Python dict (JSON-compatible).
    """
    with Image.open(image_path) as im:
        if SEMMeta is None:
            raise ImportError("SEMMeta module is required for this function.")
        
        meta, tags = SEMMeta.ImageMetadata(im)

        # 1. Map TIFF tag IDs to names
        meta_named = {TAGS.get(k, str(k)): v for k, v in dict(meta).items()}

        # 2. Parse Instrument XML metadata (tag 34682)
        instrument_meta = {}
        xml_data = dict(meta).get(34682)
        if xml_data:
            xml_data = xml_data[0]  # tuple → string
            root = ET.fromstring(xml_data)

            for data in root.findall(".//Data"):
                label = data.find("Label").text.strip()
                value = data.find("Value").text.strip()
                unit_elem = data.find("Unit")
                unit = unit_elem.text.strip() if unit_elem is not None and unit_elem.text else ""

                if unit:
                    instrument_meta[label] = {"value": value, "unit": unit}
                else:
                    instrument_meta[label] = {"value": value}

            # Replace XML string in TIFF metadata with parsed JSON
            meta_named["34682"] = instrument_meta

        # 3. Merge results
        final_meta = {
            "TIFF_Metadata": meta_named,
            "Instrument_Metadata": instrument_meta
        }

        return final_meta


def extract_metadata_json(image_path: str) -> str:
    """
    Same as extract_metadata but returns a JSON string instead of dict.
    """
    return json.dumps(extract_metadata(image_path), indent=2)


In [91]:
# Get Python dict
meta_dict = extract_metadata("Acquire BF.tif")

# Or get JSON string
json_output = extract_metadata_json("Acquire BF.tif")
print(json_output)


ImportError: SEMMeta module is required for this function.

In [87]:
# Get Python dict
meta_dict = extract_metadata("Site.tif")

# Or get JSON string
json_output = extract_metadata_json("Site.tif")
print(json_output)


ParseError: syntax error: line 1, column 0 (<string>)

In [139]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from sem_meta import SEMMeta # your existing module

def parse_tag_34682(value):
    """
    Parse TIFF tag 34682 into structured JSON-compatible data.
    """
    if not value:
        return None

    # Ensure string
    if isinstance(value, bytes):
        try:
            value = value.decode("utf-8", errors="ignore")
        except Exception:
            return {"raw": str(value)}

    # Try XML first
    try:
        root = ET.fromstring(value)
        return {root.tag: {child.tag: child.text for child in root}}
    except ET.ParseError:
        pass

    # Try plain text (ITX-style, key=value lines)
    if "=" in value:
        lines = value.splitlines()
        kv = {}
        for line in lines:
            if "=" in line:
                key, val = line.split("=", 1)
                kv[key.strip()] = val.strip()
        if kv:
            return {"plain": kv}

    # Fallback → raw string
    return {"raw": value}


def convert_meta_to_json(meta):
    """
    Convert SEMMeta metadata dict into JSON-safe dict,
    handling tag 34682 specially.
    """
    clean_meta = {}
    for k, v in meta.items():
        if k == 34682:
            # unpack tuple if necessary
            if isinstance(v, (tuple, list)):
                v = v[0]
            clean_meta[str(k)] = parse_tag_34682(v)
        else:
            clean_meta[str(k)] = list(v) if isinstance(v, (tuple, list)) else v
    return clean_meta


# Example usage with your existing code
with Image.open("Btest/image_004.tif") as im:
    meta, tags = SEMMeta.ImageMetadata(im)

    json_meta = convert_meta_to_json(meta)

    # Save JSON
    with open("image_004_metadata.json", "w", encoding="utf-8") as f:
        json.dump(json_meta, f, indent=2)

    print("✅ Metadata saved to image_004_metadata.json")


✅ Metadata saved to image_004_metadata.json


In [143]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path
from sem_meta import SEMMeta  # your existing module

def parse_tag_34682(value):
    """Parse TIFF tag 34682 into structured JSON-compatible data."""
    if not value:
        return None

    # Ensure string
    if isinstance(value, bytes):
        try:
            value = value.decode("utf-8", errors="ignore")
        except Exception:
            return {"raw": str(value)}

    # Try XML first
    try:
        root = ET.fromstring(value)
        return {root.tag: {child.tag: child.text for child in root}}
    except ET.ParseError:
        pass

    # Try plain text (ITX-style, key=value lines)
    if "=" in value:
        kv = {}
        for line in value.splitlines():
            if "=" in line:
                key, val = line.split("=", 1)
                kv[key.strip()] = val.strip()
        if kv:
            return {"plain": kv}

    # Fallback → raw string
    return {"raw": value}


def convert_meta_to_json(meta):
    """Convert SEMMeta metadata dict into JSON-safe dict, handling tag 34682 specially."""
    clean_meta = {}
    for k, v in meta.items():
        if k == 34682:
            if isinstance(v, (tuple, list)):
                v = v[0]
            clean_meta[str(k)] = parse_tag_34682(v)
        else:
            clean_meta[str(k)] = list(v) if isinstance(v, (tuple, list)) else v
    return clean_meta


# Folder containing TIFF images
input_folder = Path("Atest")

# Process each .tif file in the folder
for tif_path in input_folder.glob("*.tif"):
    try:
        with Image.open(tif_path) as im:
            meta, tags = SEMMeta.ImageMetadata(im)
            json_meta = convert_meta_to_json(meta)

            # Save JSON file with same name as TIFF
            json_path = tif_path.with_suffix(".json")
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(json_meta, f, indent=2)

            print(f"✅ Metadata saved to {json_path.name}")
    except Exception as e:
        print(f"❌ Failed to process {tif_path.name}: {e}")


✅ Metadata saved to tile_sand.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 006.json
❌ Failed to process KLE256_05spolirised (1).tif: Object of type bytes is not JSON serializable
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 007.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 005.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 011.json
✅ Metadata saved to Acquire HAADF.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 010.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 004.json
✅ Metadata saved to Acquire HAADF4.json
❌ Failed to process KLE256_05spolirised.tif: Object of type bytes is not JSON serializable
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 001.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 003.json
✅ Metadata saved to SEM Multi-Detector Image_TLD_1 - SliceImage - 002.json
✅ Metadata saved to 372Ha

In [171]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path
from sem_meta import SEMMeta  # your existing module


def parse_tag_34682(value):
    """Parse TIFF tag 34682 into structured JSON-compatible data."""
    if not value:
        return None

    if isinstance(value, bytes):
        try:
            value = value.decode("utf-8", errors="ignore")
        except Exception:
            return None

    # Try XML first
    try:
        root = ET.fromstring(value)
        return {root.tag: {child.tag: child.text for child in root}}
    except ET.ParseError:
        pass

    # Try plain key=value
    if "=" in value:
        kv = {}
        for line in value.splitlines():
            if "=" in line:
                key, val = line.split("=", 1)
                kv[key.strip()] = val.strip()
        if kv:
            return {"plain": kv}

    return None


def safe_value(v):
    """Return JSON-safe value, or None if it cannot be serialized/parsed."""
    try:
        if isinstance(v, (tuple, list)):
            return [safe_value(x) for x in v]
        if isinstance(v, bytes):
            try:
                return v.decode("utf-8", errors="ignore")
            except Exception:
                return None
        # Ensure JSON-safe
        json.dumps(v)
        return v
    except Exception:
        return None


def convert_meta_to_json(meta):
    """Convert SEMMeta metadata dict into JSON-safe dict, handling tag 34682 specially."""
    clean_meta = {}
    for k, v in meta.items():
        try:
            if k == 34682:
                if isinstance(v, (tuple, list)):
                    v = v[0]
                clean_meta[str(k)] = parse_tag_34682(v)
            else:
                clean_meta[str(k)] = safe_value(v)
        except Exception:
            clean_meta[str(k)] = None
    return clean_meta


# ===== MAIN LOOP =====
input_folder = Path("Atest")
output_folder = Path("Atest_json")  # separate folder for JSON
output_folder.mkdir(exist_ok=True)

success_count = 0
failures = []

for tif_path in input_folder.glob("*.tif"):
    try:
        with Image.open(tif_path) as im:
            meta, tags = SEMMeta.ImageMetadata(im)
            json_meta = convert_meta_to_json(meta)

            json_path = output_folder / (tif_path.stem + ".json")
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(json_meta, f, indent=2)

            print(f"✅ Metadata saved to {json_path}")
            success_count += 1
    except Exception as e:
        print(f"❌ Failed to process {tif_path.name}: {e}")
        failures.append(tif_path.name)

print("\n===== Summary =====")
print(f"✅ Succeeded: {success_count}")
print(f"❌ Failed: {len(failures)}")
if failures:
    print("Failed images:", ", ".join(failures))


✅ Metadata saved to Atest_json/tile_sand.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 006.json
✅ Metadata saved to Atest_json/KLE256_05spolirised (1).json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 007.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 005.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 011.json
✅ Metadata saved to Atest_json/Acquire HAADF.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 010.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 004.json
✅ Metadata saved to Atest_json/Acquire HAADF4.json
✅ Metadata saved to Atest_json/KLE256_05spolirised.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 001.json
✅ Metadata saved to Atest_json/SEM Multi-Detector Image_TLD_1 - SliceImage - 003.json
✅ Metadata saved to Atest_json/SEM Multi-Detector

In [175]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path
from sem_meta import SEMMeta  # your existing module


def xml_to_dict(elem):
    """Recursively convert XML element to dict."""
    if len(elem) == 0:
        text = elem.text.strip() if elem.text and elem.text.strip() else None
        return text
    result = {}
    for child in elem:
        result[child.tag] = xml_to_dict(child)
    return result


def parse_value(value):
    """
    Try to parse a metadata value into JSON-safe structure.
    - If XML → parse as nested dict
    - If key=value text → parse into dict
    - If bytes → decode to string
    - Else → return safe JSON value or None
    """
    if value is None:
        return None

    # Unpack tuples/lists
    if isinstance(value, (tuple, list)):
        return [parse_value(v) for v in value]

    # Decode bytes
    if isinstance(value, bytes):
        try:
            value = value.decode("utf-8", errors="ignore")
        except Exception:
            return None

    # Try XML
    if isinstance(value, str):
        try:
            root = ET.fromstring(value)
            return {root.tag: xml_to_dict(root)}
        except ET.ParseError:
            pass

        # Try key=value pairs
        if "=" in value:
            kv = {}
            for line in value.splitlines():
                if "=" in line:
                    key, val = line.split("=", 1)
                    kv[key.strip()] = val.strip()
            if kv:
                return {"plain": kv}

    # Ensure JSON-safe
    try:
        json.dumps(value)
        return value
    except Exception:
        return None


def convert_meta_to_json(meta):
    """Convert SEMMeta metadata dict into JSON-safe dict, parsing all values."""
    clean_meta = {}
    for k, v in meta.items():
        try:
            clean_meta[str(k)] = parse_value(v)
        except Exception:
            clean_meta[str(k)] = None
    return clean_meta


# ===== MAIN LOOP =====
input_folder = Path("Atest")
output_folder = Path("Atest_json1")
output_folder.mkdir(exist_ok=True)

success_count = 0
failures = []

for tif_path in input_folder.glob("*.tif"):
    try:
        with Image.open(tif_path) as im:
            meta, tags = SEMMeta.ImageMetadata(im)
            json_meta = convert_meta_to_json(meta)

            json_path = output_folder / (tif_path.stem + ".json")
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(json_meta, f, indent=2)

            print(f"✅ Metadata saved to {json_path}")
            success_count += 1
    except Exception as e:
        print(f"❌ Failed to process {tif_path.name}: {e}")
        failures.append(tif_path.name)

print("\n===== Summary =====")
print(f"✅ Succeeded: {success_count}")
print(f"❌ Failed: {len(failures)}")
if failures:
    print("Failed images:", ", ".join(failures))


✅ Metadata saved to Atest_json1/tile_sand.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 006.json
✅ Metadata saved to Atest_json1/KLE256_05spolirised (1).json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 007.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 005.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 011.json
✅ Metadata saved to Atest_json1/Acquire HAADF.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 010.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 004.json
✅ Metadata saved to Atest_json1/Acquire HAADF4.json
✅ Metadata saved to Atest_json1/KLE256_05spolirised.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 001.json
✅ Metadata saved to Atest_json1/SEM Multi-Detector Image_TLD_1 - SliceImage - 003.json
✅ Metadata saved to Atest_json1/SEM 

In [183]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path
from sem_meta import SEMMeta  # your existing module


def xml_to_dict(elem):
    """
    Recursively convert XML element to dict.
    Handles repeated tags as lists.
    """
    children = list(elem)
    if not children:
        text = elem.text.strip() if elem.text and elem.text.strip() else None
        return text

    result = {}
    for child in children:
        child_dict = xml_to_dict(child)
        if child.tag in result:
            # Convert to list if multiple children with same tag
            if not isinstance(result[child.tag], list):
                result[child.tag] = [result[child.tag]]
            result[child.tag].append(child_dict)
        else:
            result[child.tag] = child_dict
    return result


def parse_value(value):
    """
    Parse metadata value into JSON-safe structure.
    Supports XML, key=value strings, bytes, tuples/lists.
    """
    if value is None:
        return None

    # Unpack tuples/lists
    if isinstance(value, (tuple, list)):
        return [parse_value(v) for v in value]

    # Decode bytes
    if isinstance(value, bytes):
        try:
            value = value.decode("utf-8", errors="ignore")
        except Exception:
            return None

    # Try XML
    if isinstance(value, str):
        try:
            root = ET.fromstring(value)
            return {root.tag: xml_to_dict(root)}
        except ET.ParseError:
            pass

        # Try key=value pairs
        if "=" in value:
            kv = {}
            for line in value.splitlines():
                if "=" in line:
                    key, val = line.split("=", 1)
                    kv[key.strip()] = val.strip()
            if kv:
                return {"plain": kv}

    # Ensure JSON-safe
    try:
        json.dumps(value)
        return value
    except Exception:
        return None


def convert_meta_to_json(meta):
    """Convert SEMMeta metadata dict into JSON-safe dict, parsing all values."""
    clean_meta = {}
    for k, v in meta.items():
        try:
            clean_meta[str(k)] = parse_value(v)
        except Exception:
            clean_meta[str(k)] = None
    return clean_meta


# ===== MAIN LOOP =====
input_folder = Path("Btest")
output_folder = Path("Btest_json2")
output_folder.mkdir(exist_ok=True)

success_count = 0
failures = []

for tif_path in input_folder.glob("*.tif"):
    try:
        with Image.open(tif_path) as im:
            meta, tags = SEMMeta.ImageMetadata(im)
            json_meta = convert_meta_to_json(meta)

            json_path = output_folder / (tif_path.stem + ".json")
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(json_meta, f, indent=2)

            print(f"✅ Metadata saved to {json_path}")
            success_count += 1
    except Exception as e:
        print(f"❌ Failed to process {tif_path.name}: {e}")
        failures.append(tif_path.name)

print("\n===== Summary =====")
print(f"✅ Succeeded: {success_count}")
print(f"❌ Failed: {len(failures)}")
if failures:
    print("Failed images:", ", ".join(failures))


✅ Metadata saved to Btest_json2/CBS_5kVBD_86pA.json
✅ Metadata saved to Btest_json2/PMal2-2Lw-Ti-law.json
✅ Metadata saved to Btest_json2/highcurrent_013.json
✅ Metadata saved to Btest_json2/masks_021.json
✅ Metadata saved to Btest_json2/masks_020.json
✅ Metadata saved to Btest_json2/bat414bx5k_002.json
✅ Metadata saved to Btest_json2/BSE_image-Verios.json
✅ Metadata saved to Btest_json2/bse x200.json
✅ Metadata saved to Btest_json2/BAT405_001.json
✅ Metadata saved to Btest_json2/image_002.json
✅ Metadata saved to Btest_json2/D.json
✅ Metadata saved to Btest_json2/implant ti00.json
✅ Metadata saved to Btest_json2/C.json
✅ Metadata saved to Btest_json2/image_004.json
✅ Metadata saved to Btest_json2/A.json
✅ Metadata saved to Btest_json2/Implant 1.json
✅ Metadata saved to Btest_json2/EPMA_map-Mn.json
✅ Metadata saved to Btest_json2/TLD_5kVBD_86pA_SI_001.json
✅ Metadata saved to Btest_json2/RP179905x1200_005.json
✅ Metadata saved to Btest_json2/PP-541-29-Fram-Verios.json
✅ Metadata saved 

In [185]:
import json
import xml.etree.ElementTree as ET
from PIL import Image
from pathlib import Path
from sem_meta import SEMMeta  # your existing module


def xml_to_dict(elem):
    """Recursively convert XML element to dict, handling repeated tags as lists."""
    children = list(elem)
    if not children:
        text = elem.text.strip() if elem.text and elem.text.strip() else None
        return text

    result = {}
    for child in children:
        child_dict = xml_to_dict(child)
        tag = strip_ns_key(child.tag)
        if tag in result:
            if not isinstance(result[tag], list):
                result[tag] = [result[tag]]
            result[tag].append(child_dict)
        else:
            result[tag] = child_dict
    return result


def strip_ns_key(key):
    """Remove XML namespace from a key, e.g., '{http://ns.adobe.com/xap/1.0/}CreateDate' -> 'CreateDate'"""
    return key.split('}')[-1] if '}' in key else key


def strip_ns(d):
    """Recursively strip namespaces from dict keys."""
    if isinstance(d, dict):
        return {strip_ns_key(k): strip_ns(v) for k, v in d.items()}
    elif isinstance(d, list):
        return [strip_ns(i) for i in d]
    else:
        return d


def parse_value(value):
    """Parse metadata value into JSON-safe structure."""
    if value is None:
        return None

    # Unpack tuples/lists
    if isinstance(value, (tuple, list)):
        return [parse_value(v) for v in value]

    # Decode bytes
    if isinstance(value, bytes):
        try:
            value = value.decode("utf-8", errors="ignore")
        except Exception:
            return None

    # Try XML
    if isinstance(value, str):
        try:
            root = ET.fromstring(value)
            return {strip_ns_key(root.tag): xml_to_dict(root)}
        except ET.ParseError:
            pass

        # Try key=value pairs
        if "=" in value:
            kv = {}
            for line in value.splitlines():
                if "=" in line:
                    key, val = line.split("=", 1)
                    kv[key.strip()] = val.strip()
            if kv:
                return {"plain": kv}

    # Ensure JSON-safe
    try:
        json.dumps(value)
        return value
    except Exception:
        return None


def convert_meta_to_json(meta):
    """Convert SEMMeta metadata dict into JSON-safe dict, parsing all values."""
    clean_meta = {}
    for k, v in meta.items():
        try:
            parsed_value = parse_value(v)
            clean_meta[str(k)] = strip_ns(parsed_value)
        except Exception:
            clean_meta[str(k)] = None
    return clean_meta


# ===== MAIN LOOP =====
input_folder = Path("Btest")
output_folder = Path("Btest_json3")
output_folder.mkdir(exist_ok=True)

success_count = 0
failures = []

for tif_path in input_folder.glob("*.tif"):
    try:
        with Image.open(tif_path) as im:
            meta, tags = SEMMeta.ImageMetadata(im)
            json_meta = convert_meta_to_json(meta)

            json_path = output_folder / (tif_path.stem + ".json")
            with open(json_path, "w", encoding="utf-8") as f:
                json.dump(json_meta, f, indent=2)

            print(f"✅ Metadata saved to {json_path}")
            success_count += 1
    except Exception as e:
        print(f"❌ Failed to process {tif_path.name}: {e}")
        failures.append(tif_path.name)

print("\n===== Summary =====")
print(f"✅ Succeeded: {success_count}")
print(f"❌ Failed: {len(failures)}")
if failures:
    print("Failed images:", ", ".join(failures))


✅ Metadata saved to Btest_json3/CBS_5kVBD_86pA.json
✅ Metadata saved to Btest_json3/PMal2-2Lw-Ti-law.json
✅ Metadata saved to Btest_json3/highcurrent_013.json
✅ Metadata saved to Btest_json3/masks_021.json
✅ Metadata saved to Btest_json3/masks_020.json
✅ Metadata saved to Btest_json3/bat414bx5k_002.json
✅ Metadata saved to Btest_json3/BSE_image-Verios.json
✅ Metadata saved to Btest_json3/bse x200.json
✅ Metadata saved to Btest_json3/BAT405_001.json
✅ Metadata saved to Btest_json3/image_002.json
✅ Metadata saved to Btest_json3/D.json
✅ Metadata saved to Btest_json3/implant ti00.json
✅ Metadata saved to Btest_json3/C.json
✅ Metadata saved to Btest_json3/image_004.json
✅ Metadata saved to Btest_json3/A.json
✅ Metadata saved to Btest_json3/Implant 1.json
✅ Metadata saved to Btest_json3/EPMA_map-Mn.json
✅ Metadata saved to Btest_json3/TLD_5kVBD_86pA_SI_001.json
✅ Metadata saved to Btest_json3/RP179905x1200_005.json
✅ Metadata saved to Btest_json3/PP-541-29-Fram-Verios.json
✅ Metadata saved 